![Open Alex Logo](./images/openalex-logo.png)

## PLANTEAMIENTO Y ACOTACIÓN DE OBJETIVOS INICIALES
El propósito inicial pretendía responder a la pregunta de qué instituciones y países son los mayores divulgadores de conocimientos en cada rama del conocimiento humano.

Este objetivo era totalmente cumplible si no fuese porque openalex ya nos devolvía todos los datos de forma estructurada:

![Diagram OpenAlex components](./images/openalex-diagram.jpg)

Es decir, en este punto del proyecto carecíamos de fuentes de datos no estructuradas:

* Por ello, optamos por dirigir el proyecto a un enfoque diferente que parte del interés de comprender que tecnologías de programación son las mas usadas, en artículos o proyectos de investigación relacionados con la computación, y como varía su uso y distribución en función de cada subcampo.

Además, identificamos una problemática con el alcance definido para el proyecto (dado que se desarrollaría exclusivamente en 5h durante el Hackathon):

* OpenAlex cuenta con un total de **200 Millones de archivos**, lo cual implicaría una cantidad alarmante de tiempo para extraer y procesar toda la información necesaria para cumplir el objetivo. Por lo tanto, decidimos utilizar un conjunto de restricciones para acotar el alcance de los datos:

    * Usamos el campo _Temática_ para filtrar el número de archivos a extraer, en este aspecto decidimos filtrar una sucesión de temáticas padre-hijo relacionadas con nuestra meta: 

        ***Physical Sciences (domain) -> Computer Science (field) -> Artificial Inteligence (subfield) -> Todas las temáticas hijas de AI sin hijo (topics)***

    * Agregamos un conjunto de campos restrictivos como que el idioma en el que estén escritos sea en inglés, agregamos filtrado de keyword por lenguajes de programación en artículos y pedimos exclusivamente los articulos que tengan vinculado su pdf y además que esté sea open access. Esto redujo el número de obras a procesar a ~6000 artículos:

![Filtro temáticas dentro de OpenAlex](./images/filtro_openalex.jpeg)


Por último decidimos además simplificar mucho más las entidades a almacenar, reduciendo lo máximo posible la estructura y manteniendo la funcionalidad que se quería obtener con el objetivo. A continuación se muestra la imagen de la estructura completa de la base de datos antes y después de modificarla:

### Diagrama Entidad-Relación del sistema antes de la adaptación
![Diagrama Entidad-Relación del sistema antes](./images/relaciones%20db.png)
### Diagrama Entidad-Relación del sistema después de la adaptación
![Diagrama Entidad-Relación del sistema después](./images/esquema_redux.jpeg)

## TRABAJO REALIZADO EN EL HACKATHON

### CREACIÓN BASE DE DATOS POSTGRESQL
---


En nuestra base de datos de PostgreSQL representamos las entidades como las siguientes tablas:

* **Obra:** Engloba los artículos citados dentro de OpenAlex. Tiene dos funciones esenciales:
    * Almacenar los datos estructurados de cada artículo organizados por sus atributos: nombre, doi, url, etc.
    * Enlazar  _Tecnologia_ y _Tematica_ para poder realizar búsquedas de datos filtrando por ambas entidades. Cabe destacar que, como _Tecnologia_ y _Obra_ tienen una relación N:M, fue necesario crear una tabla para representar esta relación. Esto no pasa con la relación entre _Obra_ y _Tematica_ ya que, como es 1:N, con indicar un atributo *tematica_id* es suficiente.

* **Tecnologia:** Engloba los distintos lenguajes de programación obtenidos al extraer y analizar el contenido de los artículos. Los datos son solamente no estructurados. Está enlazado con _Obra_ por *Obra_tecnologia*.

* **Tematica:** Aquí se alamacena cada temática encontrada en la búsqueda de OpenAlex, por lo que los datos de esta tabla son estructurados. Se encuentra relacionado con _Obra_ gracias a que las obras de una temática concreta almacenan su id. Lo importante aquí es que existen temáticas que son subtemáticas de otras. La idea es que las obras estén relacionadas solamente con temáticas sin hijos para que estas a su vez se relacionen con sus padres, haciendo que todas las obras de cada hijo se relacionen con el padre. Para hacer esta relación de _Tematica_ con _Tematica_, fue necesaria la creación de una nueva tabla: *tematica_contenida*.

Además, se han creado estas tablas para las relaciones:

* **Obra_tecnologia:** Hace referencia a la relación N:M entre _Tecnologia_ y _Obra_.

* **Tematica_contenida:** Hace referencia a la relación padre a hijo de _Tematica_ con _Tematica_.

Todas estas tablas fueron creadas dentro de un *SQL script*. Para subirla a PostgreSQL, se conecta el docker, se indican los parámetros necesarios en el Python y, si no hay ningún problema, se suben.

In [2]:
import psycopg2
from psycopg2 import sql

DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}

sql_script = """CREATE TABLE IF NOT EXISTS tematica (
    id INTEGER,
    nombre_campo TEXT NOT NULL,
        PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS tecnologia (
    id INTEGER,
    nombre TEXT NOT NULL,
    tipo TEXT,
    version TEXT,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS obra (
    id INTEGER,
    doi TEXT UNIQUE,  -- NEW
    direccion_fuente TEXT NOT NULL,
    titulo TEXT NOT NULL,
    abstract TEXT,
    fecha_publicacion TEXT,
    idioma TEXT,
    num_citas INTEGER DEFAULT 0,
    fwci REAL,
    tematica_id INTEGER,
    PRIMARY KEY (id),
    FOREIGN KEY (tematica_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS obra_tecnologia (
    id INTEGER,
    obra_id INTEGER NOT NULL,
    tecnologia_id INTEGER NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (obra_id)
        REFERENCES obra(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (tecnologia_id)
        REFERENCES tecnologia(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);

CREATE TABLE IF NOT EXISTS tematica_contenida (
    id INTEGER,
    tematica_padre_id INTEGER NOT NULL,
    tematica_hijo_id INTEGER NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (tematica_padre_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (tematica_hijo_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    CHECK (tematica_padre_id <> tematica_hijo_id)
);

CREATE INDEX IF NOT EXISTS idx_obra_tematica ON obra(tematica_id);
CREATE INDEX IF NOT EXISTS idx_obratec_tecnologia ON obra_tecnologia(tecnologia_id);
CREATE INDEX IF NOT EXISTS idx_tematica_hijo ON tematica_contenida(tematica_hijo_id);
"""

def main():
    # Connect to default database to check/create demoDB
    connection = psycopg2.connect(
        host=DB_PARAMS['host'],
        port=DB_PARAMS['port'],
        database="demoDB",
        user=DB_PARAMS['user'],
        password=DB_PARAMS['password']
    )
    connection.autocommit = True
    cursor = connection.cursor()

    cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (DB_PARAMS["database"],))
    exists = cursor.fetchone()

    if not exists:
        cursor.execute(sql.SQL(f"CREATE DATABASE {DB_PARAMS['database']};"))
        print(f"✅ Database '{DB_PARAMS['database']}' created.")
    else:
        print(f"ℹ️ Database '{DB_PARAMS['database']}' already exists.")

    cursor.close()
    connection.close()

    # Connect to demoDB to create tables
    connection = psycopg2.connect(**DB_PARAMS)
    cursor = connection.cursor()
    cursor.execute(sql_script)
    connection.commit()
    cursor.close()
    connection.close()
    print("✅ Tables created or verified successfully.")

if __name__ == "__main__":
    main()

ℹ️ Database 'demoDB' already exists.
✅ Tables created or verified successfully.


### OBTENCIÓN DATOS ESTRUCTURADOS (REQUESTS & CSV)
---

La idea ahora es obtener los datos estructurados, es decir, el contenido de _Obra_, _Tematica_ y *Tematica_contenida*. Para utilizar posteriormente estos datos, se ha creado una carpeta **cache** para almacenar estas tablas ya con la información estructurada en formato csv. Para explicar el proceso completo de esta fase, podemos dividirla en las siguientes subfases:

1. Creación del csv de la entidad _obra_
2. Búsqueda y obtención de los artículos pedidos.
3. Creación del csv de la entidad _tematica_
4. Creación del csv de la relación *tematica_contenida*

En el código, se puede observar que el main sigue una estructura equivalente:

```
def main():
    initialize_csv_files()
    tematica_map = fetch_all_works()
    save_tematica_csv(tematica_map)
    update_tematica_and_generate_tematica_contenida()
```

Además, cabe destacar que se han declarado variables al inicio del código para que la búsqueda siga los filtros explicados en la introducción y para tener ya almacenadas las direcciones donde se crearan los csvs:

```
BASE_URL = "https://api.openalex.org/works"
PER_PAGE = 200
KEYWORDS = [
    "python","c-programming-language","javascript","java","java-programming-language",
    "sql","dart","swift","cobol","fortran","matlab","prolog","lisp","haskell","rust","perl",
    "scala","html","html5"
]
SUBFIELD_ID = "subfields/1702"
LANGUAGE = "languages/en"
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # go up one level from src/
CACHE_DIR = os.path.join(BASE_DIR, "cache")

CSV_OBRA = os.path.join(CACHE_DIR, "obra.csv")
CSV_TEMATICA = os.path.join(CACHE_DIR, "tematica.csv")
CSV_TEMATICA_CONTENIDA = os.path.join(CACHE_DIR, "tematica_contenida.csv")

os.makedirs(CACHE_DIR, exist_ok=True)

BASE_TOPICS = ["Physical Sciences", "Computer Science", "Artificial Intelligence"]
```
#### Creación del csv de la entidad _obra_
Crea el archivo *obra.csv* (o lo abre si ya esta creado) y crea la tabla vacía (atributos son las columnas y habrá tantas filas como obras distintas). Este proceso se encuentra represnetado dentro de la función *initialize_csv_files*.

#### Búsqueda y obtención de los artículos pedidos.
Como no todas las obras aparecen por página (límite: **200**). Este proceso se ha creado siguiendo esta lógica:
1. La búsqueda se va a realizar en bucle, por lo que es necesario crear los valores no afectados antes de empezar. Las que tienen mayor importancia son la listado/diccionario de temáticas (*tematic_map*), el id de la temática siguiente (*next_tematica_id*) y la página actual (*page*).

2. Empieza al bucle. Al inicio se realiza la llamada a OpenAlex. A esta se le pasa por parámetros la página actual (*page*), el límite de 200 (*PER_PAGE*), un filtro formado por los datos declarados previamente (*filter*) y el orden (*sort*). Solo en caso deque la función que realiza la llamada (*fetch_page*) devuelva los resultados se sigue con el proceso.

3. Saca todos los datos estrcuturados para las tres tablas, usando el identificador correspondiente de cada atributo dentro de OpenAlex. El abstract se restructura para que se pueda insertar en el csv (*reconstruct_abstract*)

4. En cuanto a la temática. Se obtiene su nombre. Si ya está en *tematic_map* se ignora. Si no, se añade al diccionario, siendo el nombre la clave y su valor el *next_tematica_id*, el cual es su nuevo id. Al valor de esta última variable se le suma uno para la siguiente ejecución.

5. Se saca *tematica_id* con *tematic_map* y se crea una nueva fila en *obra_csv* con los datos del artículo. Si el bucle ha terminado con los artículos de la página, va a la siguiente (*page* + 1) y repite el bucle. Si ya no se encuentran más obras, se finaliza.

6. Se devuelve *tematic_map* para la creación de los otros dos csv.

La función que sigue este proceso es *fetch_all_works*.

#### Creación del csv de la entidad _tematica_
Se crea el *tematica_csv* tal cual se creó el de _obra_, pero como se tienen ya el número de temáticas con sus ids gracias a *tematic_map*, se llena la tabla al completo con todas las temáticas. La función que sigue este proceso es *save_tematica_csv*.

#### Creación del csv de la relación *tematica_contenida*
Esta función se ha creado con la idea de que ya sabemos quienes son los padres: ***Physical Sciences -> Computer_Science -> Artificial Inteligence -> Temáticas sin hijo***. Por tanto, a partir de cada temática dentro de *tematica_csv*, se analiza si es una de los posibles padres. Si es una sin hijos, se declara como hija de *Artificial Inteligence* con un identificador propio (más el de *Artificial Inteligence* y el hijo). Para las que se son padres, se decalra directamente la relación a partir de la estructura antes mencionada. Con esto, se crea *tematica_contenida.csv*  y en este archivo se primero los nombres de las columnas (*id*, *id_padre* e *id_hijo*) y posteriormente tantas filas como relaciones detectadas (ids distintos), incluidas las relaciones entre los padres. La función que sigue este proceso es *update_tematica_and_generate_tematica_contenida*.

In [ ]:
import requests
import csv
import time
import os
from pathlib import Path

BASE_URL = "https://api.openalex.org/works"
PER_PAGE = 200

KEYWORDS = [
    "python","c-programming-language","javascript","java","java-programming-language",
    "sql","dart","swift","cobol","fortran","matlab","prolog","lisp","haskell","rust","perl",
    "scala","html","html5"
]
SUBFIELD_ID = "subfields/1702" #Artificial Intelligence
LANGUAGE = "languages/en"

NOTEBOOK_DIR = Path.cwd()
BASE_DIR = str(NOTEBOOK_DIR.parent)
CACHE_DIR = os.path.join(BASE_DIR, "cache")

CSV_OBRA = os.path.join(CACHE_DIR, "obra.csv")
CSV_TEMATICA = os.path.join(CACHE_DIR, "tematica.csv")
CSV_TEMATICA_CONTENIDA = os.path.join(CACHE_DIR, "tematica_contenida.csv")

os.makedirs(CACHE_DIR, exist_ok=True)

BASE_TOPICS = ["Physical Sciences", "Computer Science", "Artificial Intelligence"] # Base topics to ensure presence of hierarchy

def reconstruct_abstract(abstract_inverted_index):
    if not abstract_inverted_index or not isinstance(abstract_inverted_index, dict):
        return ""
    position_map = {}
    for word, positions in abstract_inverted_index.items():
        for pos in positions:
            position_map[pos] = word
    return " ".join(position_map[pos] for pos in sorted(position_map.keys()))

def fetch_page(url, params, max_retries=5, delay_base=2):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, params=params, timeout=30)
            if response.status_code == 200:
                return response.json()
            print(f"⚠️ Warning: Bad response {response.status_code}, retrying...")
        except Exception as e:
            print(f"⚠️ Warning: Exception during request: {e}")
        retries += 1
        time.sleep(delay_base ** retries)
    print(f"❌ Error: Failed to fetch page after {max_retries} retries.")
    return None

def initialize_csv_files():
    os.makedirs(os.path.dirname(CSV_OBRA), exist_ok=True)
    with open(CSV_OBRA, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            "id","direccion_fuente","titulo","abstract","fecha_publicacion",
            "idioma","num_citas","fwci","tematica_id","doi"
        ])
    print(f"Initialized '{CSV_OBRA}' for writing works.")

def fetch_all_works():
    tematica_map = {}
    next_tematica_id = 1
    obra_id = 1
    page = 1
    total_results = None

    while True:
        params = {
            "page": page,
            "per_page": PER_PAGE,
            "filter": f"open_access.is_oa:true,has_content.pdf:true,primary_topic.subfield.id:{SUBFIELD_ID},best_oa_location.is_accepted:true,language:{LANGUAGE},keywords.id:{'|'.join(KEYWORDS)}",
            "sort": "cited_by_count:desc"
        }
        data = fetch_page(BASE_URL, params)
        if not data or "results" not in data:
            print(f"No data returned for page {page}, stopping.")
            break

        works = data["results"]
        if total_results is None:
            total_results = data.get("meta", {}).get("count", 0)
            print(f"Total results to fetch (approximate): {total_results}")

        print(f"Fetched page {page} with {len(works)} works.")

        with open(CSV_OBRA, "a", newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for work in works:
                # --- PDF URL ---
                pdf_url = work.get("best_oa_location", {}).get("pdf_url")
                if not pdf_url:
                    continue

                # --- DOI ---
                doi = work.get("doi", "")

                titulo = work.get("title", "")
                abstract = reconstruct_abstract(work.get("abstract_inverted_index"))
                fecha_publicacion = work.get("publication_date", "")
                idioma = work.get("language", LANGUAGE)
                num_citas = work.get("cited_by_count", 0)
                fwci = work.get("fwci", "")
                primary_topic = work.get("primary_topic")
                if not primary_topic:
                    continue
                topic_name = primary_topic.get("display_name", "Unknown Topic")
                if topic_name not in tematica_map:
                    tematica_map[topic_name] = next_tematica_id
                    next_tematica_id += 1
                tematica_id = tematica_map[topic_name]

                writer.writerow([
                    obra_id, pdf_url, titulo, abstract, fecha_publicacion,
                    idioma, num_citas, fwci, tematica_id, doi
                ])
                obra_id += 1

        if page * PER_PAGE >= total_results or not works:
            break
        page += 1

    print(f"Finished fetching all works. Total works saved: {obra_id-1}")
    return tematica_map

def save_tematica_csv(tematica_map):
    os.makedirs(os.path.dirname(CSV_TEMATICA), exist_ok=True)
    with open(CSV_TEMATICA, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["id","nombre_campo"])
        for topic_name, topic_id in tematica_map.items():
            writer.writerow([topic_id, topic_name])
    print(f"Saved '{CSV_TEMATICA}' with {len(tematica_map)} topics.")

def update_tematica_and_generate_contenida():
    if not os.path.exists(CSV_TEMATICA):
        raise FileNotFoundError(f"{CSV_TEMATICA} not found.")

    tematicas = {}
    rows = []
    with open(CSV_TEMATICA, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            row["id"] = int(row["id"])
            rows.append(row)
            tematicas[row["nombre_campo"].strip()] = row["id"]

    max_id = max(r["id"] for r in rows)
    for topic in BASE_TOPICS:
        if topic not in tematicas:
            max_id += 1
            tematicas[topic] = max_id
            rows.append({"id": max_id, "nombre_campo": topic})
            print(f"Added base topic '{topic}' with id={max_id}")

    with open(CSV_TEMATICA, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["id", "nombre_campo"])
        writer.writeheader()
        writer.writerows(rows)

    relaciones = [
        {"id_padre": tematicas["Physical Sciences"], "id_hijo": tematicas["Computer Science"]},
        {"id_padre": tematicas["Computer Science"], "id_hijo": tematicas["Artificial Intelligence"]}
    ]
    ai_id = tematicas["Artificial Intelligence"]
    for nombre, id_ in tematicas.items():
        if nombre not in BASE_TOPICS:
            relaciones.append({"id_padre": ai_id, "id_hijo": id_})
    relaciones = [{"id_padre": p, "id_hijo": h} for p, h in {(r["id_padre"], r["id_hijo"]) for r in relaciones}]

    os.makedirs(os.path.dirname(CSV_TEMATICA_CONTENIDA), exist_ok=True)
    with open(CSV_TEMATICA_CONTENIDA, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["id_padre", "id_hijo"])
        writer.writeheader()
        writer.writerows(relaciones)

    print(f"'{CSV_TEMATICA}' updated with {len(rows)} topics.")
    print(f"'{CSV_TEMATICA_CONTENIDA}' generated with {len(relaciones)} relations.")

def main():
    print("Starting OpenAlex fetch process...")
    initialize_csv_files()
    tematica_map = fetch_all_works()
    save_tematica_csv(tematica_map)
    update_tematica_and_generate_contenida()
    print("Finished fetching and processing all works and topics.")

if __name__ == "__main__":
    main()

Starting OpenAlex fetch process...
Initialized 'c:\Users\kzzazzk\Downloads\gestbd-hackathon\cache\obra.csv' for writing works.
Total results to fetch (approximate): 6067
Fetched page 1 with 200 works.
Fetched page 2 with 200 works.
Fetched page 3 with 200 works.
Fetched page 4 with 200 works.
Fetched page 5 with 200 works.
Fetched page 6 with 200 works.
Fetched page 7 with 200 works.
Fetched page 8 with 200 works.
Fetched page 9 with 200 works.
Fetched page 10 with 200 works.
Fetched page 11 with 200 works.
Fetched page 12 with 200 works.
Fetched page 13 with 200 works.
Fetched page 14 with 200 works.
Fetched page 15 with 200 works.
Fetched page 16 with 200 works.
Fetched page 17 with 200 works.
Fetched page 18 with 200 works.
Fetched page 19 with 200 works.
Fetched page 20 with 200 works.
Fetched page 21 with 200 works.
Fetched page 22 with 200 works.
Fetched page 23 with 200 works.
Fetched page 24 with 200 works.
Fetched page 25 with 200 works.
Fetched page 26 with 200 works.
Fetched

### ⚠️CELDA NO EJECUTABLE PORQUE SE EJECUTÓ Y SE GENERARON LOS CSV PARA GUARDAR SUS RESULTADOS PREVIAMENTE (SE TARDARON MÁS DE 6H EN PROCESAR LOS RESULTADOS Y NO HA DADO TIEMPO NI HAY RECURSOS PARA VOLVER A EJECUTAR)

### OBTENCIÓN DATOS NO ESTRUCTURADOS 



Una vez establecidos los datos estructurados, comenzamos la tarea de extracción de lenguajes de programación usando un LLM. Para la comprensión de esta fase, la dividiré en subfases:

1.  **Extracción del texto** de los documentos PDF de los artículos.
2.  **Generación de lenguajes** de programación por artículo, enviando el texto extraído de cada obra a `gpt-5-nano`.
3.  **Guardado en caché** de este procesamiento en los archivos `tecnologia.csv` y `obra_tecnologia.csv` para no tener que volver a ejecutar el LLM en caso de pérdida de datos.


#### EXTRACCIÓN DEL TEXTO DE LOS DOCUMENTOS PDF DE LOS ARTÍCULOS
Esta tarea parece simple, pero realmente es la más compleja para no perder datos. Veamos cómo está estructurado el `obra.csv` para comprenderlo:

![Filtro temáticas dentro de OpenAlex](./images/obra-csv-screenshot.png)

Como se puede observar, hay dos campos que hacen referencia al URL:
* **`doi`**: El Digital Object Identifier, este es el identificador único del artículo.
* **`direccion_fuente`**: La *open access url* más fiable desde el punto de vista de OpenAlex.

La `direccion_fuente` va a ser la primera URL a la que consultaremos, ya que asumimos que la consulta a OpenAlex siempre devuelve un archivo PDF de *open access* (dado que la restricción en la consulta así lo define).

El flujo de extracción es el siguiente:

1.  **Intento con `direccion_fuente`**:
    * Si la URL responde con un `Content-Type` de `application/pdf`, extraemos el texto directamente usando la librería `PyMuPDF` y pasamos al siguiente paso.
    * Si la URL responde con `text/html`, significa que es una página web.

2.  **Proceso de *Scraping* (si es HTML)**:
    * Se analiza el HTML buscando clases comunes que los visores de PDF web usan para renderizar texto (ej: `[class*="textLayer"]`). Si se encuentra texto, se extrae.
    * Si no se encuentra texto, se buscan todos los enlaces `<a>` en la página que terminen en `.pdf` y se intenta descargar el primer enlace encontrado.

3.  **Proceso de *Fallback* (si todo lo anterior falla)**:
    * Si la `direccion_fuente` falla o no se encuentra un PDF, se utiliza el `doi` del artículo.
    * Se consulta la API de **Unpaywall** (`https://api.unpaywall.org/v2/{doi}`) para encontrar la mejor URL de PDF de acceso abierto (`best_oa_location`).
    * Paralelamente, se intenta construir una URL directa al portal de **ACM** (`https://dl.acm.org/doi/pdf/{doi}`), que es una fuente común.

4.  **Extracción Final**:
    * Si cualquiera de estas estrategias devuelve un archivo PDF válido, se extrae su contenido con `PyMuPDF`. Si ninguna lo logra, se marca el artículo como fallido y se continúa con el siguiente.

#### MANEJO DE ERRORES (HTTP Y "SOFT 404")
El flujo anterior está protegido contra errores de varias maneras para asegurar su robustez:

* **Errores HTTP (Timeouts y Conexión):** Toda la lógica de descarga (`requests.get`) está envuelta en un bloque `try...except` genérico. Esto captura errores de red, DNS, o si el servidor tarda demasiado en responder (definido en `PDF_TIMEOUT = 30`). Si ocurre una de estas excepciones, se registra el error y se activa inmediatamente el proceso de *fallback* (Unpaywall/ACM).

* **Errores "Soft 404" en HTML:** Este es un caso crítico. A veces, un servidor devuelve un código `200 OK` (éxito) pero la página HTML es en realidad un error ("No encontrado", "Acceso denegado", etc.).
    * Para detectar esto, si la respuesta es `text/html`, el script extrae el texto plano de la página (`soup.get_text()`) y lo busca contra una lista de indicadores de error (ej: `["not found", "error 404", "no encontrado", "access denied"]`).
    * Si encuentra alguna de estas frases, considera la página como un error, la descarta, y activa el *fallback* a Unpaywall y ACM.

* **Detección de Contenido Bloqueado (Pre-LLM):** Existe un último filtro. Incluso si se extrae texto con éxito, este podría ser inútil (ej. un *paywall* o un aviso de "habilitar cookies"). La función `analyze_text_with_gpt` revisa los primeros 300 caracteres del texto. Si detecta frases como `["enable javascript and cookies to continue", "access denied", ...]`, omite el análisis del LLM para ese artículo, evitando procesar texto basura y gastar recursos.


#### GENERACIÓN DE LENGUAJES VÍA LLM
Una vez obtenido el texto crudo del artículo, se envía al LLM (`gpt-5-nano`) para su análisis.

Se utiliza un *prompt* específico que actúa como un asistente de análisis de texto. Este *prompt* tiene reglas claras:

```
instructions = """
    You are a text analysis assistant specialized in identifying programming languages mentioned in academic or technical articles.
    Analyze the provided raw text (extracted directly from a PDF). Identify and return the main programming languages mentioned in the article (do not include frameworks, libraries, or tools).
    If a “References” or “Bibliography” section appears, ignore all text after that marker.
    Return strictly in JSON, like:
    {
        "programming_languages": ["Python", "C", "Java"]
    }
    If none found:
    {
        "programming_languages": []
    }
"""
```

#### ALMACENAMIENTO EN CACHE (CSV)
El proceso completo está orquestado por la función `process_all_obras`, que itera sobre cada artículo del `obra.csv` y aplica los pasos anteriores. Los resultados se guardan para asegurar la persistencia de los datos:

* **`tecnologia.csv`**: Este archivo actúa como una tabla maestra para las tecnologías. Cuando el LLM devuelve un lenguaje (ej: "Python"), el script comprueba (con `load_tecnologias`) si "Python" ya existe en este CSV. Si no existe, se le asigna un nuevo `id` (`next_tecn_id`) y se añade al final.
* **`obra_tecnologia.csv`**: Este archivo es la tabla de unión (relación N-a-N). Por cada lenguaje identificado en un artículo, se guarda una fila que conecta el `obra_id` con el `tecnologia_id` correspondiente (ej: `[id_enlace, 101, 1]` para vincular la obra 101 con Python (id 1)).

Este sistema de caché es crucial, ya que permite reanudar el proceso en caso de error sin tener que volver a consumir recursos de la API del LLM sobre artículos ya procesados.

In [11]:
import csv
import os
import io
import requests
import json
import openai
import subprocess
from bs4 import BeautifulSoup
import fitz 
from pathlib import Path

MODEL_NAME = "mistral:instruct"
PDF_TIMEOUT = 30
UNPAYWALL_EMAIL = "your_email@example.com"
NOTEBOOK_DIR = Path.cwd()
BASE_DIR = str(NOTEBOOK_DIR.parent)
CACHE_DIR = os.path.join(BASE_DIR, "cache")

OBRAS_CSV = os.path.join(CACHE_DIR, "obra.csv")
TECN_CSV = os.path.join(CACHE_DIR, "tecnologia.csv")
OBRA_TECN_CSV = os.path.join(CACHE_DIR, "obra_tecnologia.csv")


instructions = """You are a text analysis assistant specialized in identifying programming languages mentioned in academic or technical articles.
Analyze the provided raw text (extracted directly from a PDF). Identify and return the main programming languages mentioned in the article (do not include frameworks, libraries, or tools).
If a “References” or “Bibliography” section appears, ignore all text after that marker.
Return strictly in JSON, like:
{
  "programming_languages": ["Python", "C", "Java"]
}
If none found:
{
  "programming_languages": []
}
"""

os.makedirs(CACHE_DIR, exist_ok=True)

# ----------------------
# CSV helpers
# ----------------------
def read_obras_from_csv(file_path):
    obras = []
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            obras.append((int(row["id"]), row["direccion_fuente"], row.get("doi")))
    return obras

def init_csv(file_path, headers=None):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        with open(file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if headers:
                writer.writerow(headers)
        return 1
    max_id = 0
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        try:
            reader = csv.DictReader(f)
            for row in reader:
                val = row.get("id")
                if val:
                    try:
                        max_id = max(max_id, int(val))
                    except:
                        continue
        except:
            f.seek(0)
            for line in f:
                parts = line.split(",")
                if parts:
                    try:
                        max_id = max(max_id, int(parts[0].strip()))
                    except:
                        continue
    return max_id + 1

def append_unique_to_csv(file_path, row, headers=None, key_index=1):
    init_csv(file_path, headers=headers)
    existing_keys = set()
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        peek = next(reader, None)
        if headers and peek and all(h in peek for h in headers):
            pass
        else:
            if peek:
                try:
                    existing_keys.add(peek[key_index])
                except:
                    pass
        for r in reader:
            try:
                existing_keys.add(r[key_index])
            except:
                continue
    key = row[key_index] if len(row) > key_index else None
    if key not in existing_keys:
        with open(file_path, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)



def append_to_csv(file_path, row, headers=None):
    """Simple append without uniqueness (for obra_tecnologia)"""
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists and headers:
            writer.writerow(headers)
        writer.writerow(row)
def load_tecnologias(file_path):
    tech_map = {}
    if os.path.exists(file_path):
        with open(file_path, newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                tech_map[row["nombre"]] = int(row["id"])
    return tech_map

# ----------------------
# PDF + Analysis
# ----------------------
def get_text_from_pdf_url(pdf_url, doi=None):
    tried_urls = set()
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    def fetch_unpaywall_pdf(doi):
        if not doi:
            return None
        try:
            unpaywall_url = f"https://api.unpaywall.org/v2/{doi}?email={UNPAYWALL_EMAIL}"
            r = requests.get(unpaywall_url, timeout=10)
            if r.status_code == 200:
                data = r.json()
                pdf_link = data.get("best_oa_location", {}).get("url_for_pdf")
                if pdf_link:
                    print(f"📖 Found Unpaywall PDF: {pdf_link}")
                    return pdf_link
        except Exception as e:
            print(f"⚠️ Unpaywall fetch failed: {e}")
        return None


def get_text_from_pdf_url(pdf_url, doi=None):
    tried_urls = set()
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    def fetch_unpaywall_pdf(doi):
        if not doi:
            return None
        try:
            unpaywall_url = f"https://api.unpaywall.org/v2/{doi}?email={UNPAYWALL_EMAIL}"
            r = requests.get(unpaywall_url, timeout=10)
            if r.status_code == 200:
                data = r.json()
                pdf_link = data.get("best_oa_location", {}).get("url_for_pdf")
                if pdf_link:
                    print(f"📖 Found Unpaywall PDF: {pdf_link}")
                    return pdf_link
        except Exception as e:
            print(f"⚠️ Unpaywall fetch failed: {e}")
        return None

    def fetch_acm_pdf(doi):
        if not doi:
            return None
        try:
            acm_url = f"https://dl.acm.org/doi/pdf/{doi}"
            r = requests.head(acm_url, allow_redirects=True, timeout=10)
            if r.status_code == 200 and "pdf" in r.headers.get("Content-Type", "").lower():
                print(f"📄 Found ACM PDF: {acm_url}")
                return acm_url
        except Exception as e:
            print(f"⚠️ ACM fetch failed: {e}")
        return None

    while pdf_url and pdf_url not in tried_urls:
        tried_urls.add(pdf_url)
        try:
            response = requests.get(pdf_url, headers=headers, timeout=PDF_TIMEOUT)
            content_type = response.headers.get("Content-Type", "").lower()

            if "application/pdf" in content_type:
                try:
                    pdf_bytes = io.BytesIO(response.content)
                    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
                    text = "\n\n".join([page.get_text() for page in doc])
                    if not text.strip():
                        raise ValueError("No text extracted from PDF")
                    return text.strip(), pdf_url
                except Exception as e:
                    print(f"⚠️ PDF parse error with PyMuPDF: {e}")
                pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                continue

            if "text/html" in content_type:
                soup = BeautifulSoup(response.text, "html.parser")
                body_text = soup.get_text(separator=' ', strip=True).lower()
                if any(x in body_text for x in ["not found", "error 404", "no encontrado", "access denied"]):
                    pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                    continue
                text_labels = soup.select('[class*="textLayer"], [id*="textLayer"] div, span')
                texts = [el.get_text(separator=' ', strip=True) for el in text_labels]
                if texts:
                    return " ".join(texts), pdf_url
                pdf_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
                if pdf_links:
                    next_pdf = requests.compat.urljoin(pdf_url, pdf_links[0])
                    if next_pdf not in tried_urls:
                        pdf_url = next_pdf
                        continue
                pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                continue

            pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)

        except Exception as e:
            print(f"⚠️ Exception while fetching PDF: {e}")
            pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)

    print("❌ No valid PDF or text found.")
    return None, None



def estimate_tokens(text: str) -> int:
    """Estimate the number of tokens in a string for GPT models."""
    return len(ENCODING.encode(text))

def analyze_text(instructions, pdf_text):
    detected_languages = set()

    # 1️⃣ Try LLM first
    if pdf_text.strip():
        prompt = f"{instructions}\n\nText:\n{pdf_text}"
        try:
            result = subprocess.run(
                ["ollama", "run", MODEL_NAME],
                input=prompt,
                capture_output=True,
                text=True,
                encoding="utf-8",
                errors="ignore"
            )
            raw = result.stdout.strip()
            json_start = raw.find("{")
            json_end = raw.rfind("}") + 1
            if json_start != -1 and json_end != -1:
                llm_result = json.loads(raw[json_start:json_end])
                detected_languages.update(llm_result.get("programming_languages", []))
        except Exception:
            pass

    return {"programming_languages": sorted(detected_languages)}

# Make sure to set your API key in the environment
# export OPENAI_API_KEY="sk-..."
def analyze_text_with_gpt(pdf_text, model="gpt-5-nano"):
    """
    Analyze PDF text using ChatGPT Responses API.
    Returns a set of detected programming languages.
    Automatically skips blocked content.
    """
    detected_languages = set()

    # Blocked content check
    blocked_indicators = [
        "enable javascript and cookies to continue",
        "access denied",
        "not found",
        "error 404"
    ]
    preview_text = pdf_text[:300].replace("\n", " ").lower()
    if any(b in preview_text for b in blocked_indicators):
        print("⚠️ Blocked content detected, skipping analysis.")
        return {"programming_languages": []}

    # GPT-5 request
    try:
        response = openai.responses.create(
            model=model,
            input=f"""
            You are a text analysis assistant specialized in identifying programming languages mentioned in academic or technical articles.

            Task:
            1️⃣ Identify **only actual programming languages** used to write code.
            2️⃣ Do **NOT** include frameworks, libraries, standards, formal languages, or platforms.
            3️⃣ Ignore any text after a “References” or “Bibliography” section.
            4️⃣ Return strictly in JSON:

            {{"programming_languages": ["Python", "C", "Java"]}}

            If none found, return:

            {{"programming_languages": []}}


            Text:
            {pdf_text}
"""
        )
        raw = response.output_text.strip()
        json_start = raw.find("{")
        json_end = raw.rfind("}") + 1
        if json_start != -1 and json_end != -1:
            llm_result = json.loads(raw[json_start:json_end])
            detected_languages.update(llm_result.get("programming_languages", []))
    except Exception as e:
        print(f"⚠️ GPT analysis failed: {e}")

    return {"programming_languages": sorted(detected_languages)}

# ----------------------
# Main loop
# ----------------------
def process_all_obras():
    obras = read_obras_from_csv(OBRAS_CSV)
    print(f"Found {len(obras)} obras in CSV.")

    next_tecn_id = init_csv(TECN_CSV, headers=["id","nombre"])
    next_link_id = init_csv(OBRA_TECN_CSV, headers=["id","obra_id","tecnologia_id"])

    for obra_id, pdf_url, doi in obras:
        print(f"\n🔹 Processing Obra ID: {obra_id}")
        try:
            # Step 1: fetch PDF text
            text, final_url = get_text_from_pdf_url(pdf_url, doi)
            if not text:
                print(f"❌ No valid PDF or text found.")
                print(f"⚠️ Skipping Obra ID {obra_id}, no text extracted.")
                continue
            else:
                preview = text[:300].replace("\n", " ").strip()
                print(f"📄 Text extracted for Obra ID {obra_id} ({len(text)} chars)")
                print(f"🔗 Source URL used: {final_url}")
                print(f"📝 Text preview: {preview}{'...' if len(text) > 300 else ''}")

            # Step 2: analyze with Ollama
            print(f"🤖 Analyzing text for Obra ID {obra_id}...")
            try:
                result = analyze_text_with_gpt(text)
            except Exception as e:
                print(f"⚠️ Analysis failed for Obra ID {obra_id}: {e}")
                result = {"programming_languages": []}

            languages = result.get("programming_languages", [])
            print(f"📝 Obra ID {obra_id} languages detected: {languages}")

            tech_map = load_tecnologias(TECN_CSV)  # { "Python": 89, "C": 90, ... }

            for lang in languages:
                if lang not in tech_map:
                    tech_map[lang] = next_tecn_id
                    append_to_csv(TECN_CSV, [next_tecn_id, lang], headers=["id","nombre"])
                    next_tecn_id += 1

                tecnologia_id = tech_map[lang]
                append_to_csv(OBRA_TECN_CSV, [next_link_id, obra_id, tecnologia_id], headers=["id","obra_id","tecnologia_id"])
                next_link_id += 1

        except Exception as e:
            print(f"⚠️ Unexpected error processing Obra ID {obra_id}: {e}")


if __name__ == "__main__":
    #Uncomment if you are testing runs 
    """for csv_file in [TECN_CSV, OBRA_TECN_CSV]:
        if os.path.exists(csv_file):
            os.remove(csv_file)
            print(f"🗑️ Deleted old CSV: {csv_file}")"""
    process_all_obras()


Found 6010 obras in CSV.

🔹 Processing Obra ID: 1
❌ No valid PDF or text found.
❌ No valid PDF or text found.
⚠️ Skipping Obra ID 1, no text extracted.

🔹 Processing Obra ID: 2
❌ No valid PDF or text found.
❌ No valid PDF or text found.
⚠️ Skipping Obra ID 2, no text extracted.

🔹 Processing Obra ID: 3
❌ No valid PDF or text found.
❌ No valid PDF or text found.
⚠️ Skipping Obra ID 3, no text extracted.

🔹 Processing Obra ID: 4
📄 Text extracted for Obra ID 4 (1903 chars)
🔗 Source URL used: https://joss.theoj.org/papers/10.21105/joss.00024.pdf
📝 Text preview: corner.py: Scatterplot matrices in Python Daniel Foreman-Mackey1 1 Sagan Fellow, University of Washington DOI: 10.21105/joss.00024 Software • Review • Repository • Archive Licence Authors of JOSS papers retain copyright and release the work un- der a Creative Commons Attri- bution 4.0 International...
🤖 Analyzing text for Obra ID 4...


KeyboardInterrupt: 

### ALMACENAMIENTO EN POSTGRESQL
---

Una vez declaradas las tablas en **PostgreSQL**, y los datos obtenidos mediante **OpenAlex**, y la API de **OpenAI**, y guardados en archivos *.csv (en la carpeta **cache**), procedemos a trasladar todos los datos guardados en dichos archivos y almacenarlos en nuestra base de datos.

Primero nos conectamos a la base de datos y obtenemos un cursor, mediante la biblioteca `psycopg2-binary`.
Segundo cargamos directamente todos los archivos de cache como `DataFrames` de `pandas`.

`df_tematica = pd.read_csv(file_tematica)`

Tabla a tabla, vamos leyendo cada fila de su `DataFrame` e insertandolo en su tabla correspondiente mediante **SQL**. Por ejemplo: el archivo tematica.csv sirve para rellenar la tabla tematica de la base de datos. En caso de conflito por datos repetidos, la orden de insercion se ignora.

```
for _, row in df_tematica.iterrows():
        cursor.execute("""
            INSERT INTO tematica (id, nombre_campo)
            VALUES (%s, %s)
            ON CONFLICT (id) DO NOTHING
        """, row['id'], row['nombre_campo']))
```

Una vez todos los datos de cache han sido cargados, guardamos los cambios desconectamos el cursor y cerramos la conexion.

`connection.commit()`

In [6]:
import os
import pandas as pd
import psycopg2
from pathlib import Path
DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}

def main():
    connection = psycopg2.connect(**DB_PARAMS)
    cursor = connection.cursor()

    notebook_dir = Path.cwd()
    script_dir = str(notebook_dir.parent)
    dir_cache = os.path.join(script_dir, "cache")


    # File paths
    file_tematica = os.path.join(dir_cache, 'tematica.csv')
    file_tematica_contenida = os.path.join(dir_cache, 'tematica_contenida.csv')
    file_obra = os.path.join(dir_cache, 'obra.csv')
    file_tecnologia = os.path.join(dir_cache, 'tecnologia.csv')
    file_obra_tecnologia = os.path.join(dir_cache, 'obra_tecnologia.csv')

    # Read CSVs
    df_tematica = pd.read_csv(file_tematica)
    df_tematica_contenida = pd.read_csv(file_tematica_contenida)
    df_obra = pd.read_csv(file_obra)
    df_tecnologia = pd.read_csv(file_tecnologia)
    df_obra_tecnologia = pd.read_csv(file_obra_tecnologia)

    # Insert tematica
    for _, row in df_tematica.iterrows():
        cursor.execute("""
            INSERT INTO tematica (id, nombre_campo)
            VALUES (%s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (int(row['id']), row['nombre_campo'].strip() if pd.notna(row['nombre_campo']) else None))

    # Insert tematica_contenida
    for _, row in df_tematica_contenida.iterrows():
        cursor.execute("""
            INSERT INTO tematica_contenida (id, tematica_padre_id, tematica_hijo_id)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING
        """, (int(row['id']), int(row['id_padre']), int(row['id_hijo'])))

    # Insert obra
    for _, row in df_obra.iterrows():
        cursor.execute("""
            INSERT INTO obra (
                id, doi, direccion_fuente, titulo, abstract, fecha_publicacion,
                idioma, num_citas, fwci, tematica_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (
            int(row['id']),
            row.get('doi').strip() if pd.notna(row.get('doi')) else None,
            row['direccion_fuente'].strip() if pd.notna(row.get('direccion_fuente')) else None,
            row['titulo'].strip() if pd.notna(row.get('titulo')) else None,
            row.get('abstract').strip() if pd.notna(row.get('abstract')) else None,
            row.get('fecha_publicacion') if pd.notna(row.get('fecha_publicacion')) else None,
            row.get('idioma').strip() if pd.notna(row.get('idioma')) else None,
            int(row.get('num_citas', 0)) if pd.notna(row.get('num_citas')) else 0,
            float(row.get('fwci', 0.0)) if pd.notna(row.get('fwci')) else 0.0,
            int(row.get('tematica_id')) if pd.notna(row.get('tematica_id')) else None
        ))

    # Insert tecnologia
    for _, row in df_tecnologia.iterrows():
        cursor.execute("""
            INSERT INTO tecnologia (id, nombre, tipo, version)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (
            int(row['id']),
            row['nombre'].strip() if pd.notna(row['nombre']) else None,
            row.get('tipo').strip() if pd.notna(row.get('tipo')) else None,
            row.get('version').strip() if pd.notna(row.get('version')) else None
        ))

    # Insert obra_tecnologia
    for _, row in df_obra_tecnologia.iterrows():
        cursor.execute("""
            INSERT INTO obra_tecnologia (id, obra_id, tecnologia_id)
            VALUES (%s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (int(row['id']), int(row['obra_id']), int(row['tecnologia_id'])))
# Puede que on conflict falle por lo de id 
    connection.commit()
    cursor.close()
    connection.close()
    print("✅ CSV data loaded successfully including tecnologia and obra_tecnologia.")

if __name__ == "__main__":
    main()


✅ CSV data loaded successfully including tecnologia and obra_tecnologia.


### Creación del Grafo: Turtle, Schema.org & SKOS

#### Mapeo de Entidades (PostgreSQL → Vocabularios)

En la última entrega (Práctica 3) se definió un conjunto de mapeos entre las tablas de la base de datos _PostgreSQL_ y las clases de los vocabularios **SCHEMA** y **SKOS**.  
Estos mapeos permiten representar las entidades relacionales (_obra_, _temática_, _tecnología_, _obra_tecnología_ y _temática_contenida_) como recursos semánticos dentro de un grafo RDF.

#### Generación del Grafo RDF

El siguiente paso consiste en **transformar los datos de la base de datos en un grafo RDF** y **exportarlos al formato Turtle (.ttl)**.  
Este archivo podrá ser cargado posteriormente en **GraphDB**, donde se podrán ejecutar consultas **SPARQL** sobre la información enlazada.

El proceso se realiza mediante un script en **Python** que:

1. Conecta con la base de datos PostgreSQL.  
2. Itera sobre cada fila de las tablas relevantes.  
3. Genera triples RDF utilizando las ontologías **SCHEMA** y **SKOS**.  
4. Exporta el resultado final a un archivo `.ttl`.

#### Descripción del Script

El script realiza los siguientes pasos principales:

- **Conexión a la base de datos** PostgreSQL mediante `psycopg2`.  
- **Creación de un grafo RDF** usando la librería `rdflib`.  
- **Asignación de namespaces**:  
  - `schema:` → [https://schema.org/](https://schema.org/)  
  - `skos:` → [https://www.w3.org/2004/02/skos/core#](https://www.w3.org/2004/02/skos/core#)  
  - `openalex:` → [https://openalex.org/](https://openalex.org/)  
- **Iteración sobre las tablas**:
  - `tematica` → Clases `skos:Concept`  
  - `tematica_contenida` → Relaciones jerárquicas `skos:broader` y `skos:narrower`  
  - `tecnologia` → Clases `schema:SoftwareApplication`  
  - `obra` → Clases `schema:TechArticle`  
  - `obra_tecnologia` → Relaciones `schema:mentions`  
- **Exportación del grafo** en formato Turtle (`.ttl`)  
- **Cierre de la conexión** a la base de datos.

#### Resultado Final

El script genera un archivo RDF llamado **`openalex_graph.ttl`** dentro del directorio `ttl/`.  
Este archivo contiene todos los triples RDF generados a partir de la base de datos y puede cargarse directamente en **GraphDB**.

Una vez cargado, es posible ejecutar consultas **SPARQL** para:

- Explorar relaciones entre obras y tecnologías.  
- Visualizar jerarquías temáticas mediante SKOS.  
- Analizar la estructura semántica del conocimiento extraído.

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import psycopg2
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, SKOS, XSD

# --- Namespaces ---
SCHEMA = Namespace("https://schema.org/")
OPENALEX = Namespace("https://openalex.org/")
g = Graph()
g.bind("schema", SCHEMA)
g.bind("skos", SKOS)
g.bind("openalex", OPENALEX)

# --- PostgreSQL connection ---
DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}


conn = psycopg2.connect(**DB_PARAMS)

cur = conn.cursor()

print("Connected to database ✅")

# --- 1. TEMÁTICA ---
cur.execute("SELECT id, nombre_campo FROM tematica;")
for tmid, nombre in cur.fetchall():
    tema_uri = OPENALEX[f"tematica_{tmid}"]
    g.add((tema_uri, RDF.type, SKOS.Concept))
    g.add((tema_uri, SKOS.prefLabel, Literal(nombre)))

print("Mapped table: tematica ✅")

# tematica_contenida → skos:broader
cur.execute("SELECT id, tematica_padre_id, tematica_hijo_id FROM tematica_contenida;")
for tcid, parent, child in cur.fetchall():
    parent_uri = OPENALEX[f"tematica_{parent}"]
    child_uri = OPENALEX[f"tematica_{child}"]
    g.add((parent_uri, SKOS.narrower, child_uri))
    g.add((child_uri, SKOS.broader, parent_uri))

# --- 2. TECNOLOGÍA ---
cur.execute("SELECT id, nombre, tipo, version FROM tecnologia;")
for tid, nombre, tipo, version in cur.fetchall():
    tech_uri = OPENALEX[f"tecnologia_{tid}"]
    g.add((tech_uri, RDF.type, SCHEMA.SoftwareApplication))
    g.add((tech_uri, SCHEMA.name, Literal(nombre)))
    if tipo:
        g.add((tech_uri, SCHEMA.applicationCategory, Literal(tipo)))
    if version:
        g.add((tech_uri, SCHEMA.softwareVersion, Literal(version)))

print("Mapped table: tecnologia ✅")

# --- 3. OBRA ---
cur.execute("SELECT id, doi, direccion_fuente, titulo, abstract, fecha_publicacion, idioma, num_citas, fwci, tematica_id FROM obra;")
for oid, doi, direccion_fuente, titulo, abstract, fecha_publicacion, idioma, num_citas, fwci, tematica_id in cur.fetchall():
    obra_uri = OPENALEX[f"obra_{oid}"]
    g.add((obra_uri, RDF.type, SCHEMA.TechArticle))
    if doi:
        g.add((obra_uri, SCHEMA.sameAs, Literal(doi)))
    if direccion_fuente:
        g.add((obra_uri, SCHEMA.url, Literal(direccion_fuente)))
    if titulo:
        g.add((obra_uri, SCHEMA.name, Literal(titulo)))
    if abstract:
        g.add((obra_uri, SCHEMA.abstract, Literal(abstract)))
    if fecha_publicacion:
        g.add((obra_uri, SCHEMA.datePublished, Literal(fecha_publicacion, datatype=XSD.date)))
    if idioma:
        g.add((obra_uri, SCHEMA.inLanguage, Literal(idioma)))
    if num_citas:
        g.add((obra_uri, SCHEMA.citationCount, Literal(num_citas, datatype=XSD.integer)))
    if fwci:
        g.add((obra_uri, SCHEMA.metric, Literal(fwci, datatype=XSD.float)))
    if tematica_id:
        g.add((obra_uri, SCHEMA.about, OPENALEX[f"tematica_{tematica_id}"]))

print("Mapped table: obra ✅")

# obra_tecnologia → schema:mentions
cur.execute("SELECT obra_id, tecnologia_id FROM obra_tecnologia;")
for oid, tid in cur.fetchall():
    g.add((OPENALEX[f"obra_{oid}"], SCHEMA.mentions, OPENALEX[f"tecnologia_{tid}"]))


print("Mapped relationships ✅")

# --- 4. EXPORT ---
output_file = "../ttl/openalex_graph.ttl"
g.serialize(destination=output_file, format="turtle")
print(f"RDF graph exported to {output_file} 🧩")

# --- 5. Cleanup ---
cur.close()
conn.close()
print("PostgreSQL connection closed 🔒")


Connected to database ✅
Mapped table: tematica ✅
Mapped table: tecnologia ✅
Mapped table: obra ✅
Mapped relationships ✅
RDF graph exported to ../ttl/openalex_graph.ttl 🧩
PostgreSQL connection closed 🔒


### CREACIÓN DE CONSULTAS
---

Para finalizar el proyecto, hemos creado una serie de querys con las que se pueda analizar sí el sistema cumple los objetivos programados.

Primeramente, se probó si cumplía el objetivo principal: obtener el número de apariciones de distintos lenguajes de programación en obras de cada subtemática (o temática sin hijas) distinta. El resultado fue una tabla donde se listaban todas las combinaciones de tópico y lenguaje distinto dentro de la base de datos, junto al número de veces que se repetía esa relación. Por lo tanto, se cumplió la meta descrita al inicio del proyecto.

Para demostrar su funcionamiento, solo hay que utilizar la siguiente consulta en GraphDB:

In [ ]:
"""
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

# Queremos una lista de "aristas" (conexiones)
SELECT ?topicName ?techName (COUNT(DISTINCT ?work) AS ?sharedWorksCount)
WHERE {
    ?work schema:about ?topic .
    ?work schema:mentions ?technology .
    
    OPTIONAL { ?topic skos:prefLabel ?topicName . }
    OPTIONAL { ?technology schema:name ?techName . }
    
    FILTER(BOUND(?topicName) && BOUND(?techName))
}
GROUP BY ?topicName ?techName
ORDER BY DESC(?sharedWorksCount)

"""

Posteriormente, para exprimir los límites del sistema, decidimos probar una consulta más: Tecnologías más competitivas entre campos. En esta consulta, se lista el número de veces que cada par de tecnologías distintas (por ejemplo, Python y Java, C++ y Prolog, etc.) aparecen en las mismas obras. Con el resultado, aprendimos que Python y Java eran la dupla que más aparecía en las mismas obras.

Para demostrar su funcionamiento, solo hay que utilizar la siguiente consulta en GraphDB:

In [ ]:
"""
PREFIX schema: <https://schema.org/>

SELECT ?techA_name ?techB_name (COUNT(DISTINCT ?topic) AS ?commonTopics)
WHERE {
    # Encuentra la primera tecnología (A) mencionada por una obra
    ?workA schema:mentions ?techA .
    ?workA schema:about ?topic .
    ?techA schema:name ?techA_name .

    # Encuentra la segunda tecnología (B) mencionada por la misma obra (o una obra sobre el mismo tema)
    ?workB schema:mentions ?techB .
    ?workB schema:about ?topic . # <-- Mismo tema
    ?techB schema:name ?techB_name .

    # Asegura que no sea la misma tecnología
    FILTER (?techA != ?techB)

    # Solo queremos los nombres, no las URIs largas
    FILTER(BOUND(?techA_name) && BOUND(?techB_name))
}
GROUP BY ?techA_name ?techB_name
ORDER BY DESC(?commonTopics)
LIMIT 10

"""

### FUTUROS INCREMENTOS
---
Actualmente, en la tabla de tecnologías almacenamos cerca de 600 filas. Al ver la tabla más de cerca, en realidad, muchas de las tecnologías utilizadas no son mas que frameworks o derivaciones de ya existentes, por ejemplo, si se busca el lenguaje **Java** en la lista, aparece cerca de 10 veces (sin contar **javascript**): Java, Java-K, Featherweight Java, etc.; y solo Java es un lenguaje de programacion, los demas son derivaciones que usan Java como base. Por degracia, nuestro LLM encargado de encontrar cada tecnologia usada no otorga el lenguaje principal en el que se basa el obtenido, asi que hemos terminado una lista de 600 tecnologías, que seria interesante simplificar para ajustarnos a nuestros objetivos.

Proponemos como solución, reducir tabla de relaciones, eleiminando y sustiuyendo aquellas tecnologías que no sean lenguajes de programación, por aquellas de las que deriva, por ejemplo, sustituir Java-K por Java; y propagar los cambios a la tabla obras y actualizar las relaciones antiguas por las nuevas.

Para ello, identificaremos primero cuales tecnologías son lenguajes de programación y cuales no, utilizando la base de datos con API **Wikidata**, que tiene clase titulada `programming language`. Una vez obtenido cuales son lenguajes de programación, proponemos varios métodos, para determinar la herencia entre tecnologías.

El primer metodo consisten en utilizar embeddings para transformar las tecnologías, y aplicar el algoritmo K-Means para detectar los grupos que se forman entorno a cada lenguaje. Sin embargo, este algoritmo puede presentar varios problemas, siendo uno de ellos, que múltiples lenguajes de programación acaben en un mismo cluster, y otras tecnologías aparezcan aisladas en un grupo aparte.

Otra solución que proponemos, es utilizar de nuevo embeddings, pero en esta vez utilizar el algoritmo K-Nearest Neighbors (K-NN), que teniendo en cuenta cuales son los lenguajes de programación, organize cada tecnología no identificada como lenguaje alrededor de una que si lo es.

### PARTICIPANTES:
* Jaime Vaquero Rabahieh. Correo: jaime.vaquero@alumnos.upm.es
* Zakaria Lasry Sahraoui. Correo: z.lsahraoui@alumnos.upm.es
* Damian Sanchez Maqueda. Correo: damian.sanchez@alumnos.upm.es
* Radu-Andrei Bourceanu. Correo: r.bourceanu@alumnos.upm.es